In [1]:
import pandas as pd
import os
import numpy as np

import re
import pprint

import nltk
nltk.download('vader_lexicon')

# For sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

# Parts of speech
nltk.download('averaged_perceptron_tagger')

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

#from imblearn.over_sampling import SMOTE

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
data = pd.read_csv("aita_features.csv")

In [17]:
# Have to have entirely numeric features
# Change edited to 0 or 1
# change pos/neg to 0 or 1
# remove id, title, body, 0, 0.1
data['edited'] = data['edited'].apply(lambda c: 0 if c == 'False' else 1)
data['pos_neg'] = data['pos_neg'].apply(lambda c: 1 if c == 'pos' else 0)
# remove id, title, body

In [18]:
# Split data

data["gendered"] = "n"
for index,row in data.iterrows():
    if pd.notnull(data.at[index,"age"]):
        if pd.notnull(data.at[index,"gender"]):
            data.at[index,"gendered"] = "y"
            
gendered = data[data["gendered"] == "y"]
not_gendered = data[data["gendered"] == "n"]
del gendered["gendered"]
del not_gendered["gendered"]

In [19]:
y_gendered = gendered["verdict"]
del gendered["verdict"]
X_gendered = gendered

y_not_gendered = not_gendered["verdict"]
del not_gendered["verdict"]
X_not_gendered = not_gendered

In [20]:
X_traindev_gendered, X_test_gendered, y_traindev_gendered, y_test_gendered = train_test_split(X_gendered, y_gendered, test_size=0.10, random_state=224)
# Use these vvv
X_train_gendered, X_dev_gendered, y_train_gendered, y_dev_gendered = train_test_split(X_traindev_gendered, y_traindev_gendered, test_size=0.1111, random_state=224)

X_traindev_not_gendered, X_test_not_gendered, y_traindev_not_gendered, y_test_not_gendered = train_test_split(X_not_gendered, y_not_gendered, test_size=0.10, random_state=224)
# Use these vvv
X_train_not_gendered, X_dev_not_gendered, y_train_not_gendered, y_dev_not_gendered = train_test_split(X_traindev_not_gendered, y_traindev_not_gendered, test_size=0.1111, random_state=224)

In [43]:
### End splitting
len(X_train_not_gendered)
X_dev_gendered

,Unnamed: 0,Unnamed: 0.1,id,timestamp,title,body,edited,score,num_comments,is_asshole,...,scores,compound,pos_neg,proper_nouns,cardinals,existential_there,personal_pronouns,question_words,negations,sentence_length
96917,97004,97004,ew4izv,1.580386e+09,WIBTA for unfriending friends I have know for ...,I (f24) have always been friends with a close ...,0,109,31.0,0,...,"{'neg': 0.087, 'neu': 0.844, 'pos': 0.068, 'co...",-0.8978,0,4,12,0,47,3,6,21.058824
56851,56934,56934,ctgh4m,1.566392e+09,AITA for expecting my partner to be ready when...,I (33M) have to pick up my partner (32F) a lot...,0,110,72.0,0,...,"{'neg': 0.024, 'neu': 0.902, 'pos': 0.074, 'co...",0.8900,1,3,9,1,26,8,2,15.000000
56090,56173,56173,crwpz6,1.566102e+09,AITA for saying no when my friend asked me to ...,"I (27F) have a friend, let’s call him Jeff (29...",0,23,16.0,0,...,"{'neg': 0.131, 'neu': 0.782, 'pos': 0.087, 'co...",-0.9785,0,24,4,0,62,6,5,28.136364
86462,86549,86549,eej8mb,1.577099e+09,WIBTA if I ask my roommate to move out? I (26f...,\n\nWe have lived together for 2 months now a...,0,3,14.0,0,...,"{'neg': 0.121, 'neu': 0.771, 'pos': 0.109, 'co...",-0.8886,0,2,13,1,100,7,11,23.766667
69428,69513,69513,dhsc23,1.571068e+09,WIBTA If I Didn't Go See My Ill Grandmother?,Just recently today (Thanksgiving here in Cana...,0,11,12.0,0,...,"{'neg': 0.112, 'neu': 0.836, 'pos': 0.052, 'co...",-0.9526,0,7,7,0,38,5,11,17.190476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62186,62270,62270,d3cst3,1.568316e+09,AITA for backing out of getting an apartment w...,This has been gnawing at me for a while. Last ...,0,118,33.0,0,...,"{'neg': 0.036, 'neu': 0.903, 'pos': 0.061, 'co...",0.9267,1,6,7,0,60,5,4,21.482759
75680,75765,75765,du3z32,1.573344e+09,WIBTA for demanding my parents pay for my kitt...,I know it probably sounds super entitled but b...,0,3,29.0,1,...,"{'neg': 0.034, 'neu': 0.865, 'pos': 0.101, 'co...",0.9599,1,2,3,0,27,0,5,18.928571
16086,16155,16155,avmiuy,1.551326e+09,AITA for asking my girlfriend to not go to a s...,So my girlfriend (22f) and I (22m) were talkin...,1,3,31.0,0,...,"{'neg': 0.058, 'neu': 0.835, 'pos': 0.107, 'co...",0.9616,1,3,2,0,46,3,8,26.800000
82173,82260,82260,e7hsvc,1.575743e+09,AITA for not wanting to invite my dad to my we...,Backstory: My parents were divorced when I was...,0,3,12.0,0,...,"{'neg': 0.062, 'neu': 0.849, 'pos': 0.089, 'co...",0.8986,1,8,7,0,47,6,7,16.681818


In [48]:
# https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
# For reporting stats!

# All functions used for running regressions

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Pass in names of vars to select, returns reduced versions of the x's
def selectVars(x_train, x_test, var_names):
    x_train_new = x_train[var_names]
    x_test_new = x_test[var_names]
    return x_train_new, x_test_new

# Get all variable names from df, except for verdict
def getVariables(df):
    features = list(data.columns)
    features.remove('verdict')
    return features

# Takes the datasets, returns predictions and model
def runRegression(x_train, y_train, x_test, y_test):
    # Create and fit model
    logreg = LogisticRegression()
    logreg.fit(x_train, y_train)
    # Get predictions
    y_pred = logreg.predict(x_test)
    return y_pred, logreg # Returns predictions, then model
def getAccuracy(logreg, x_test, y_test):
    # Print out accuracy and other stats
    #print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))
    return logreg.score(x_test, y_test)
def getClassification(y_test, y_pred):
    #print(classification_report(y_test, y_pred))
    return classification_report(y_test, y_pred)

def getROCCurve(logreg, x_test, y_test, y_pred):
    logit_roc_auc = roc_auc_score(y_test, logreg.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()
    return plt

In [49]:
# Takes x/y train and names of vars to test
# Returns a dictionary with the features, and the various metrics plus a plot of ROC
def runSingleRegression(x_train, x_test, y_train, y_test, var_names):
    # Store the results of the run
    results = {}
    # Select only the variables
    x_train, x_test = selectVars(x_train, x_test, var_names)
    # Record the names of features included
    results['features'] = list(x_train.columns)
    # Run the regression
    y_pred, logreg = runRegression(x_train, y_train, x_test, y_test)
    # get some of the stats
    results['accuracy'] = getAccuracy(logreg, x_test, y_test)
    results['metrics'] = getClassification(y_test, y_pred)
    #results['roc'] = getROCCurve(logreg, x_test, y_test, y_pred)
    return results

In [62]:
# Ablated models, without each feature individually
def tryAllFeatures(x_train, x_test, y_train, y_test, features_list):
    # Store regression results
    all_regressions = {}
    # Get all the features
    #all_features = list(x_train.columns)
    # For each feature, train a logistic regression without it
    for i in range(0, len(features_list)):
        # Drop one variable
        variables = features_list[0:i] + features_list[(i+1):] # maybe
        print(variables)
        print(len(variables))
        all_regressions[features_list[i]] = runSingleRegression(x_train, x_test, y_train, y_test, variables)
    # Train one regression 
    all_regressions['full'] = runSingleRegression(x_train, x_test, y_train, y_test, features_list)
    return all_regressions

In [63]:
# Find actual good logistic regression

# Add one that uses gender and another that doesn't

In [64]:
#all_feats = list(X_train_not_gendered.columns)
#X_train_not_gendered
# Select: 'edited' 'score' 'num_comments' 'is_asshole' 'compound' 'pos_neg', 'proper_nouns' 'cardinals' 'existential_there' 'personal_pronouns' 'question_words' 'negations' 'sentence_length' 

In [65]:
# Full list of variables to use in train:
x_features_list = ['edited','score','num_comments','compound','pos_neg', 'proper_nouns','cardinals','existential_there','personal_pronouns','question_words','negations','sentence_length'] 

all_regressions = tryAllFeatures(X_train_not_gendered, X_dev_not_gendered, y_train_not_gendered, y_dev_not_gendered, x_features_list)

['score', 'num_comments', 'compound', 'pos_neg', 'proper_nouns', 'cardinals', 'existential_there', 'personal_pronouns', 'question_words', 'negations', 'sentence_length']
11


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

['edited', 'num_comments', 'compound', 'pos_neg', 'proper_nouns', 'cardinals', 'existential_there', 'personal_pronouns', 'question_words', 'negations', 'sentence_length']
11


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

['edited', 'score', 'compound', 'pos_neg', 'proper_nouns', 'cardinals', 'existential_there', 'personal_pronouns', 'question_words', 'negations', 'sentence_length']
11


...

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

['edited', 'score', 'num_comments', 'compound', 'pos_neg', 'proper_nouns', 'cardinals', 'existential_there', 'personal_pronouns', 'question_words', 'sentence_length']
11


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

['edited', 'score', 'num_comments', 'compound', 'pos_neg', 'proper_nouns', 'cardinals', 'existential_there', 'personal_pronouns', 'question_words', 'negations']
11


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

In [66]:
all_regressions

{'edited': {'features': ['score',
   'num_comments',
   'compound',
   'pos_neg',
   'proper_nouns',
   'cardinals',
   'existential_there',
   'personal_pronouns',
   'question_words',
   'negations',
   'sentence_length'],
  'accuracy': 0.6117913832199546,
  'metrics': '                  precision    recall  f1-score   support\n\n         asshole       0.58      0.04      0.07      1925\n  everyone sucks       0.00      0.00      0.00       498\nno assholes here       0.00      0.00      0.00      1037\n not the asshole       0.61      0.99      0.76      5360\n\n        accuracy                           0.61      8820\n       macro avg       0.30      0.26      0.21      8820\n    weighted avg       0.50      0.61      0.48      8820\n'},
 'score': {'features': ['edited',
   'num_comments',
   'compound',
   'pos_neg',
   'proper_nouns',
   'cardinals',
   'existential_there',
   'personal_pronouns',
   'question_words',
   'negations',
   'sentence_length'],
  'accuracy': 0.607369